# Create daily global netcdfs


In [1]:
# --- Core data handling and plotting libraries ---
import earthaccess
import xarray as xr       # for working with labeled multi-dimensional arrays
import numpy as np        # for numerical operations on arrays
import pandas as pd
import matplotlib.pyplot as plt  # for creating plots
import cartopy.crs as ccrs
import sklearn
# --- Custom python functions ---
import os, importlib
# Looks to see if you have the file already and if not, downloads from GitHub
if not os.path.exists("ml_utils.py"):
    !wget -q https://raw.githubusercontent.com/fish-pace/2025-tutorials/main/ml_utils.py

import ml_utils as mu
importlib.reload(mu)
# core stats
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error
from scipy.ndimage import uniform_filter1d

In [2]:
!pip install google-cloud-storage

  Using cached google_cloud_storage-3.7.0-py3-none-any.whl.metadata (14 kB)
  Using cached google_auth-2.43.0-py2.py3-none-any.whl.metadata (6.6 kB)
  Using cached google_api_core-2.28.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached google_cloud_core-2.5.0-py3-none-any.whl.metadata (3.1 kB)
  Using cached google_resumable_media-2.8.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached google_crc32c-1.7.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
  Using cached googleapis_common_protos-1.72.0-py3-none-any.whl.metadata (9.4 kB)
  Using cached protobuf-6.33.2-cp39-abi3-manylinux2014_x86_64.whl.metadata (593 bytes)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
Using cached google_cloud_storage-3.7.0-py3-none-any.whl (303 kB)
Using cached goo

## Read in model

In [3]:
## Load model
bundle = mu.load_ml_bundle("models/brt_chla_profiles_bundle.zip")
brt_models = bundle.model
meta = bundle.meta
rrs_cols = meta["rrs_cols"]
chl_cols = meta["y_col"]
extra = meta["extra_cols"]
dataset = bundle.data["dataset"]
train_idx = bundle.data["train_idx"]
test_idx = bundle.data["test_idx"]
X_train = bundle.data["X_train"]
X_test = bundle.data["X_test"]
y_train_all = bundle.data["y_train"]
y_test_all = bundle.data["y_test"]


Loaded ML bundle from: models/brt_chla_profiles_bundle.zip
  model_kind : pickle
  model_type : collection (dict), n_submodels=20
  example key: CHLA_0_10
  target     : log10_CHLA_A_B depth bins
  features   : 174 columns
  train/test : 4408 / 1102 rows
  dataset    : 5510 rows stored in bundle

Usage example (Python):
  bundle = load_ml_bundle('path/to/bundle.zip')
  # Predict using helper 'predict_all_depths_for_day'
  # Example: predict all depths for one day from a BRF dataset R
  pred = bundle.predict(
      R_dataset,                  # xr.DataArray/xr.Dataset with lat/lon + predictors
      brt_models=bundle.model,    # dict of models by depth bin
      feature_cols=bundle.meta['feature_cols'],
      consts={'solar_hour': 12.0, 'type': 1},
  )  # -> e.g. CHLA(time?, z, lat, lon)

  # Plot using helper 'make_plot_pred_map'
  fig, ax = bundle.plot(pred_da, pred_label='Prediction')



## Create a dataset with our derived variables

In [4]:
import xarray as xr
import numpy as np

def build_chla_profile_dataset(CHLA: xr.DataArray) -> xr.Dataset:
    """
    Given CHLA(time, z, lat, lon), compute derived metrics and
    return an xr.Dataset suitable for writing to Zarr/NetCDF.
    """

    # Start from CHLA's own dataset so its coords (including z_start/z_end) win
    ds = CHLA.to_dataset(name="CHLA")

    # ---- Layer thickness (z dimension) ----
    z_start = CHLA.coords.get("z_start", None)
    z_end   = CHLA.coords.get("z_end", None)

    if (z_start is not None) and (z_end is not None):
        z_thick = (z_end - z_start).rename("z_thickness")   # (z)
    else:
        # fallback: uniform layer thickness, e.g. 10 m
        z_thick = xr.full_like(CHLA["z"], 10.0).rename("z_thickness")

    z_center = CHLA["z"]

    # total CHLA in column (used for validity + center-of-mass)
    col_total = CHLA.sum("z")          # (time, lat, lon)
    valid = col_total > 0              # True where there is some CHLA

    # ---- Integrated CHLA (nominal 0–200 m; actual range = z extent) ----
    CHLA_int = (CHLA * z_thick).sum("z")
    CHLA_int = CHLA_int.where(valid)
    CHLA_int.name = "CHLA_int_0_200"

    # ---- Peak value and depth (NaN-safe) ----
    CHLA_filled = CHLA.fillna(-np.inf)
    peak_idx = CHLA_filled.argmax("z")       # (time, lat, lon) integer indices

    CHLA_peak = CHLA.isel(z=peak_idx).where(valid)
    CHLA_peak.name = "CHLA_peak"

    CHLA_peak_depth = z_center.isel(z=peak_idx).where(valid)
    CHLA_peak_depth.name = "CHLA_peak_depth"

    # ---- Depth-weighted mean depth (center of mass) ----
    num = (CHLA * z_center).sum("z")
    den = col_total
    depth_cm = (num / den).where(valid)
    depth_cm.name = "CHLA_depth_center_of_mass"

    # ---- Attach derived fields to the dataset ----
    ds["CHLA_int_0_200"] = CHLA_int
    ds["CHLA_peak"] = CHLA_peak
    ds["CHLA_peak_depth"] = CHLA_peak_depth
    ds["CHLA_depth_center_of_mass"] = depth_cm
    ds["z_thickness"] = z_thick

    # ---- Variable attributes ----
    # CHLA itself should already have attrs from the prediction step
    ds["CHLA"].attrs.setdefault("units", "mg m-3")
    ds["CHLA"].attrs.setdefault("long_name", "Chlorophyll-a concentration")
    ds["CHLA"].attrs.setdefault(
        "description",
        "BRT-derived chlorophyll-a profiles from PACE hyperspectral Rrs",
    )

    ds["CHLA_int_0_200"].attrs.update(
        units="mg m-2",
        long_name="Depth-integrated chlorophyll-a",
        description=(
            "Vertical integral of CHLA over the available depth bins "
            "(nominally 0–200 m; actual range defined by z_start/z_end)."
        ),
    )

    ds["CHLA_peak"].attrs.update(
        units="mg m-3",
        long_name="Peak chlorophyll-a concentration in the water column",
        description="Maximum CHLA value over depth at each (time, lat, lon).",
    )

    ds["CHLA_peak_depth"].attrs.update(
        units="m",
        long_name="Depth of peak chlorophyll-a",
        positive="down",
        description=(
            "Depth (bin center) where CHLA is maximal in the water column "
            "at each (time, lat, lon)."
        ),
    )

    ds["CHLA_depth_center_of_mass"].attrs.update(
        units="m",
        long_name="Chlorophyll-a depth center of mass",
        positive="down",
        description=(
            "Depth of the chlorophyll-a center of mass, computed as "
            "sum_z(CHLA * z) / sum_z(CHLA)."
        ),
    )

    ds["z_thickness"].attrs.update(
        units="m",
        long_name="Layer thickness",
        description=(
            "Thickness of each vertical bin used for depth integration. "
            "Derived from z_end - z_start when available; otherwise set to a "
            "uniform nominal thickness."
        ),
    )

    # You can still add global ds.attrs later in your pipeline
    return ds

## Test pipeline bits

Get data and make prediction, build dataset. Save a test file to gcs.

In [89]:
# Get some Rrs data
import earthaccess
import xarray as xr

day = "2024-07-08"

auth = earthaccess.login()
# are we authenticated?
if not auth.authenticated:
    # ask for credentials and persist them in a .netrc file
    auth.login(strategy="interactive", persist=True)

# Get Rrs
rrs_results = earthaccess.search_data(
    short_name = "PACE_OCI_L3M_RRS",
    temporal = (day, day),
    granule_name="*.DAY.*.4km.nc"
)
f = earthaccess.open(rrs_results[0:1], pqdm_kwargs={"disable": True})
rrs_ds = xr.open_dataset(f[0])

lat_min, lat_max = 20, 40
lon_min, lon_max = -70, -60
# Get Rrs for that box
R = rrs_ds["Rrs"].sel(
    lat=slice(lat_max, lat_min),   # decreasing lat coord: max→min
    lon=slice(lon_min, lon_max)
)
pred = bundle.predict(
      R,                  # xr.DataArray/xr.Dataset with lat/lon + predictors
      brt_models=bundle.model,    # dict of models by depth bin
      feature_cols=bundle.meta['feature_cols'],
      consts={'solar_hour': 0, 'type': 1},
  )

Starting 0 of 480
Starting 100 of 480
Starting 200 of 480
Starting 300 of 480
Starting 400 of 480
Starting wrapping
Adding coords


In [ ]:
test_ds = build_chla_profile_dataset(pred)
test_ds

In [98]:
# Test saving file to the bucket
# stop annoying warnings
# https://console.cloud.google.com/storage/browser/nmfs_odp_nwfsc/CB/fish-pace-datasets
import warnings
warnings.filterwarnings("ignore", message="Your application has authenticated using end user credentials")

from google.cloud import storage
from pathlib import Path

# === Set these ===
bucket_name = "nmfs_odp_nwfsc"

# Create client and bucket
client = storage.Client(project="noaa-gcs-public-data")
bucket = client.bucket(bucket_name)

# Set the file you want to test with
test_file = Path("test.pdf")  # change this if using a different file
destination_prefix = "CB/fish-pace-datasets/chla-z/netcdf"

# Create blob and upload
blob_path = f"{destination_prefix}/{test_file.name}"
blob = bucket.blob(blob_path)
blob.upload_from_filename(str(test_file))

print(f"Uploaded {test_file.name} → gs://{bucket_name}/{blob_path}")

Uploaded test.pdf → gs://nmfs_odp_nwfsc/CB/fish-pace-datasets/chla-z/netcdf/test.pdf


## Full pipeline

* dataset to netcdf in google bucket
* Zarr

In [4]:
# Get Rrs data
import earthaccess
import xarray as xr

auth = earthaccess.login()
# are we authenticated?
if not auth.authenticated:
    # ask for credentials and persist them in a .netrc file
    auth.login(strategy="interactive", persist=True)

# Get Rrs
rrs_results = earthaccess.search_data(
    short_name = "PACE_OCI_L3M_RRS",
    granule_name="*.DAY.*.4km.nc"
)
len(rrs_results)

560

In [ ]:
%%time
import xarray as xr
import numpy as np
import pandas as pd
from pathlib import Path
import earthaccess
from google.cloud import storage
import tempfile

# Get Rrs data
import earthaccess
import xarray as xr

auth = earthaccess.login()
# are we authenticated?
if not auth.authenticated:
    # ask for credentials and persist them in a .netrc file
    auth.login(strategy="interactive", persist=True)

# Get Rrs
rrs_results = earthaccess.search_data(
    short_name = "PACE_OCI_L3M_RRS",
    granule_name="*.DAY.*.4km.nc"
)

# === Set these ===
bucket_name = "nmfs_odp_nwfsc"
# Create client and bucket
client = storage.Client(project="noaa-gcs-public-data")
bucket = client.bucket(bucket_name)
destination_prefix = "CB/fish-pace-datasets/chla-z/netcdf"

lat_chunk = 100
lon_chunk = 100

# just test on first two granules
for res in rrs_results[0:1]:
    # day as ISO string from UMM
    day_iso = res["umm"]["TemporalExtent"]["RangeDateTime"]["BeginningDateTime"]
    day = pd.to_datetime(day_iso)          # Timestamp
    day_str = day.strftime("%Y%m%d")
    print(f"Processing {day_str}...")

    # auth per granule (you can optimize this later if you want)
    auth = earthaccess.login()
    files = earthaccess.open([res], pqdm_kwargs={"disable": True})

    # open dataset
    rrs_ds = xr.open_dataset(files[0])
    # debugging line
    #rrs_ds = rrs_ds.sel(lat=slice(40, 20), lon=slice(-70, -60) )

    try:
        # Rrs for that day
        # (if the file has only one time, squeeze; otherwise select)
        if "time" in rrs_ds.dims:
            R = rrs_ds["Rrs"].sel(time=day).squeeze("time")
        else:
            R = rrs_ds["Rrs"]
        R = R.transpose("lat", "lon", "wavelength")

        # BRT predictions for all depths
        pred = bundle.predict(
            R,
            brt_models=bundle.model,
            feature_cols=bundle.meta["feature_cols"],
            consts={"solar_hour": 0, "type": 1},
            chunk_size_lat=100,
            time=day.to_datetime64(),   # time coord length 1
            z_name="z",
            silent=False,
        )  # (time=1, z, lat, lon), float32

        # Build full dataset with integrated/peak metrics
        ds_day = build_chla_profile_dataset(pred)

        # Add metadata
        ds_day["CHLA"].attrs.update(
            units="mg m-3",
            long_name="Chlorophyll-a concentration",
            description="BRT-derived CHLA profiles from PACE hyperspectral Rrs",
        )
        ds_day["z"].attrs.update(units="m", long_name="depth (bin center)")
        ds_day["lat"].attrs.update(units="degrees_north")
        ds_day["lon"].attrs.update(units="degrees_east")
        ds_day.attrs["source"] = "BRT model trained on BGC-Argo + OOI matchups"
        ds_day.attrs["model_bundle"] = Path("path/to/bundle.zip").name

        # Chunking for NetCDF
        encoding = {
            "CHLA": {
                "dtype": "float32",
                "zlib": True,
                "complevel": 4,
                "chunksizes": (1, ds_day.sizes["z"], lat_chunk, lon_chunk),
            }
            # you can add encodings for derived vars later if desired
        }

        # 4. Write to a local temporary NetCDF, then upload to GCS
        tmp_dir = Path(tempfile.gettempdir())
        local_path = tmp_dir / f"chla_z_{day_str}.nc"
        
        ds_day.to_netcdf(
            local_path,
            engine="h5netcdf",
            encoding=encoding,
        )
        
        blob_path = f"{destination_prefix}/chla_z_{day_str}.nc"
        blob = bucket.blob(blob_path)
        blob.upload_from_filename(str(local_path))
        
        print(f"Wrote {local_path} → gs://{bucket_name}/{blob_path}")
        
    finally:
        rrs_ds.close()
        del ds_day, pred, R, rrs_ds


In [3]:
import xarray as xr
import gcsfs

token = "/home/jovyan/.config/gcloud/application_default_credentials.json"
fs = gcsfs.GCSFileSystem(token=token)

url = "gcs://nmfs_odp_nwfsc/CB/fish-pace-datasets/chla-z/netcdf/chla_z_20240401.nc"
ds1 = xr.open_dataset(url, engine="h5netcdf", storage_options={"token": token})
ds1

<xarray.Dataset> Size: 4GB
Dimensions:                    (time: 1, z: 20, lat: 4320, lon: 8640)
Coordinates:
  * time                       (time) datetime64[ns] 8B 2024-04-01
  * lat                        (lat) float32 17kB 89.98 89.94 ... -89.94 -89.98
  * lon                        (lon) float32 35kB -180.0 -179.9 ... 179.9 180.0
  * z                          (z) float32 80B 5.0 15.0 25.0 ... 185.0 195.0
    z_start                    (z) float32 80B ...
    z_end                      (z) float32 80B ...
Data variables:
    CHLA                       (time, z, lat, lon) float32 3GB ...
    CHLA_int_0_200             (time, lat, lon) float32 149MB ...
    CHLA_peak                  (time, lat, lon) float32 149MB ...
    CHLA_peak_depth            (time, lat, lon) float32 149MB ...
    CHLA_depth_center_of_mass  (time, lat, lon) float32 149MB ...
    z_thickness                (z) float32 80B ...

In [ ]:
import xarray as xr
import gcsfs

token = "/home/jovyan/.config/gcloud/application_default_credentials.json"
fs = gcsfs.GCSFileSystem(token=token)

pattern = "nmfs_odp_nwfsc/CB/fish-pace-datasets/chla-z/netcdf/chla_z_*.nc"
paths = ["gcs://" + p for p in sorted(fs.glob(pattern))]
print("files:", len(paths), "first:", paths[0])

ds = xr.open_mfdataset(
    paths,
    engine="h5netcdf",
    combine="by_coords",   # uses the time coordinate in each file
    parallel=True,
    chunks={},             # don't guess chunks yet; we'll rechunk explicitly
)
ds

In [ ]:
ds = ds.chunk({
    "time": 7,
    "z": -1,
    "lat": 300,
    "lon": 300,
})

In [ ]:
zarr_path = "gcs://nmfs_odp_nwfsc/CB/fish-pace-datasets/chla-z/zarr"

ds.to_zarr(
    zarr_path,
    mode="w",
    consolidated=True,
    storage_options={"token": token},
)
print("Zarr uploaded:", zarr_path)

In [2]:
!pip install gcsfs

  Using cached gcsfs-2025.12.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached fsspec-2025.12.0-py3-none-any.whl.metadata (10 kB)
  Using cached google_auth-2.43.0-py2.py3-none-any.whl.metadata (6.6 kB)
  Using cached google_auth_oauthlib-1.2.3-py3-none-any.whl.metadata (3.1 kB)
  Using cached google_cloud_storage-3.7.0-py3-none-any.whl.metadata (14 kB)
  Using cached google_cloud_storage_control-1.8.0-py3-none-any.whl.metadata (10 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached google_auth-2.41.1-py2.py3-none-any.whl.metadata (6.6 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached google_api_core-2.28.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached google_cloud_core-2.5.0-py3-none-any.whl.metadata (3.1 kB)
  Using cached google_resumable_media-2.8.0-py3-none-any.whl.metada

In [5]:
import gcsfs

token = "/home/jovyan/.config/gcloud/application_default_credentials.json"
fs = gcsfs.GCSFileSystem(token=token)

prefix = "nmfs_odp_nwfsc/CB/fish-pace-datasets/chla-z/netcdf/"
files = fs.find(prefix)  # fast-ish prefix walk
files = [f for f in files if f.split("/")[-1].startswith("chla_z_") and f.endswith(".nc")]

paths = ["gcs://" + f for f in sorted(files)]
print("files:", len(paths), "first:", paths[0])


files: 560 first: gcs://nmfs_odp_nwfsc/CB/fish-pace-datasets/chla-z/netcdf/chla_z_20240305.nc


In [2]:
from pathlib import Path
import xarray as xr
import gcsfs
import zarr

token = "/home/jovyan/.config/gcloud/application_default_credentials.json"
fs = gcsfs.GCSFileSystem(token=token)

bucket = "nmfs_odp_nwfsc"
prefix = "CB/fish-pace-datasets/chla-z/netcdf"
pattern = f"{bucket}/{prefix}/chla_z_*.nc"
paths = sorted(fs.glob(pattern))
paths = ["gcs://" + p for p in paths]
print("nfiles:", len(paths), "first:", paths[0])

zarr_path = f"gcs://{bucket}/CB/fish-pace-datasets/chla-z/zarr_v2"

tmp_dir = Path("/tmp/chla_nc_stream")
tmp_dir.mkdir(parents=True, exist_ok=True)

for i, gcs_url in enumerate(paths[0:1], 1):
    local_nc = tmp_dir / Path(gcs_url).name

    # Download whole file (single big transfer)
    fs.get(gcs_url, str(local_nc))

    # Open locally (fast)
    ds = xr.open_dataset(local_nc, engine="h5netcdf")

    # Ensure consistent time dtype
    if "time" in ds.coords:
        ds = ds.assign_coords(time=ds["time"].astype("datetime64[ns]"))

    # Rechunk for Zarr (tune lat/lon later)
    chunk_spec = {}
    if "time" in ds.dims: chunk_spec["time"] = 1
    if "z" in ds.dims:    chunk_spec["z"] = ds.sizes["z"]
    if "lat" in ds.dims:  chunk_spec["lat"] = 256
    if "lon" in ds.dims:  chunk_spec["lon"] = 256
    ds = ds.chunk(chunk_spec)

    mode = "w" if i == 1 else "a"

    ds.to_zarr(
        zarr_path,
        mode=mode,
        append_dim="time" if mode == "a" else None,
        consolidated=False,   # consolidate once at end
        zarr_version=2,
        storage_options={"token": token},
    )

    ds.close()
    local_nc.unlink(missing_ok=True)

    print(f"wrote {i}/{len(paths)}")

# consolidate metadata once at end
fs2 = gcsfs.GCSFileSystem(token=token)
store = zarr.storage.FSStore(zarr_path, fs=fs2)
zarr.consolidate_metadata(store)
print("done + consolidated")


nfiles: 557 first: gcs://nmfs_odp_nwfsc/CB/fish-pace-datasets/chla-z/netcdf/chla_z_20240305.nc


/tmp/ipykernel_256/1873378135.py:44: FutureWarning: zarr_version is deprecated, use zarr_format
  ds.to_zarr(


wrote 1/557
done + consolidated


In [6]:
ds.sizes

Frozen({'lat': 4320, 'lon': 8640, 'z': 20, 'time': 1})